### <b>Import Packages</b>

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

### <b>Import Data</b>

In [2]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {'data_name': """Test_fields_100821_results.csv""", 'interaction_properties': {}}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

df1 = itcfs.read_pandas_and_concat(readClient, flightInfo)
df1.head()

,HALF,DOWNNUM,YARDSTOGO,SCOREDIFF,TIMELEFT,DOWNNAME,TRUEFIELD,YARDSGAINED_Mean,AVG_OVER_TOGO,PLAYIDSTR,RUN_PASS,CONCEPT_8,CONCEPT_9,pred_TD,prob_TD_NO,prob_TD_YES,pred_FIRST,prob_FIRST_NO,prob_FIRST_YES
0,0,0,5,0,480,1st,35,6.36,1.272,100000020,RUN,2,0,N,0.989180609,0.010819391,N,0.562406343,0.437593657
1,0,0,9,-7,1050,1st,45,4.756098,0.528455333,100000054,RUN,5,4,N,0.999278473,0.000721527,Y,0.439295772,0.560704228
2,0,2,4,7,600,3rd,5,8.164835,2.04120875,200000049,PASS,4,9,N,0.929719852,0.070280148,Y,0.391968242,0.608031758
3,0,0,11,7,180,1st,45,6.216667,0.565151545,200000041,PASS,4,1,N,0.999941376,5.86E-05,N,0.676802873,0.323197127
4,0,0,1,0,60,1st,45,13.416667,13.416667,200000085,PASS,8,5,N,0.800382789,0.199617211,N,0.60849475,0.39150525


In [3]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [4]:
wslib.download_file("mytest_function.py")

{'file_name': 'mytest_function.py',
 'summary': ['loaded data', 'saved to file']}

In [5]:
from mytest_function import transform_mydata

In [6]:
data = df1.copy()

df2 = transform_mydata(data)
df2.head()

,YARDSTOGO,SCOREDIFF,TIMELEFT,TRUEFIELD,CONCEPT_8,CONCEPT_9,YARDSGAINED_Mean,AVG_OVER_TOGO,prob_FIRST,TARGET_1ST,...,PLAYID_RUN1,PLAYID_RUN2,PLAYID_RUN3,PLAYID_RUN5,TEE_SHORT,TEE_MED,TEE_LONG,TEE_XLONG,DOWNNAME_1st,DOWNNAME_3rd
0,5,0,480,35,2,0,6.360000,1.272000,0.437594,0,...,0,1,0,0,0,1,0,0,1,0
1,9,-7,1050,45,5,4,4.756098,0.528455,0.560704,1,...,0,0,0,1,0,0,1,0,1,0
2,4,7,600,5,4,9,8.164835,2.041209,0.608032,1,...,0,0,0,0,0,1,0,0,0,1
3,11,7,180,45,4,1,6.216667,0.565152,0.323197,0,...,0,0,0,0,0,0,0,1,1,0
4,1,0,60,45,8,5,13.416667,13.416667,0.391505,0,...,0,0,0,0,1,0,0,0,1,0


In [7]:
import os
import requests
import json
import time
from project_lib import Project

token = os.environ.get('USER_ACCESS_TOKEN')
project_id = os.environ.get('PROJECT_ID')
host = os.environ['RUNTIME_ENV_APSX_URL']
token

'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IklTQWdYV2czTE5uRVVETVg0OFB2b1BnVEEybHpCYmhxaGNaZUEyN2d5X00ifQ.eyJ1aWQiOiIxMDAwMzMxMDMwIiwidXNlcm5hbWUiOiJkYW4uc2NoZXVjaEB1cy5pYm0uY29tIiwicm9sZSI6IlVzZXIiLCJwZXJtaXNzaW9ucyI6WyJjcmVhdGVfcHJvamVjdCIsImF1dGhvcl9nb3Zlcm5hbmNlX2FydGlmYWN0cyIsInZpZXdfZ292ZXJuYW5jZV9hcnRpZmFjdHMiLCJjYW5fcHJvdmlzaW9uIiwiYWNjZXNzX2NhdGFsb2ciLCJ2aWV3X3F1YWxpdHkiLCJtYW5hZ2VfaW5mb3JtYXRpb25fYXNzZXRzIiwibWFuYWdlX21ldGFkYXRhX2ltcG9ydCIsIm1hbmFnZV9kaXNjb3ZlcnkiLCJtYW5hZ2VfcXVhbGl0eSIsImNyZWF0ZV9zcGFjZSIsImFjY2Vzc19hZHZhbmNlZF9nb3Zlcm5hbmNlX2NhcGFiaWxpdGllcyIsImFjY2Vzc19hZHZhbmNlZF9tYXBwaW5nX2NhcGFiaWxpdGllcyIsIm1hbmFnZV9jYXRhbG9nIiwiYWNjZXNzX2luZm9ybWF0aW9uX2Fzc2V0cyIsIm1hbmFnZV9zcGFjZSIsIm1vbml0b3Jfc3BhY2UiLCJtYW5hZ2VfY2F0ZWdvcmllcyIsInZpZXdfcGxhdGZvcm1faGVhbHRoIiwibWFuYWdlX3Byb2plY3QiLCJtb25pdG9yX3Byb2plY3QiLCJtYW5hZ2Vfc2VydmljZV9pbnN0YW5jZXMiLCJtYW5hZ2VfZ292ZXJuYW5jZV93b3JrZmxvdyIsIm1vbml0b3JfcGxhdGZvcm0iXSwiZ3JvdXBzIjpbMTAwMDBdLCJzdWIiOiJkYW4uc2NoZXVjaEB1cy5pYm0uY29t

In [8]:
export_filename = 'Transformed_Data.csv'

export = df2.copy()
project = Project(project_id=project_id, project_access_token=token)
project.save_data(export_filename, export.to_csv(index=False), overwrite=True)

{'file_name': 'Transformed_Data.csv',
 'message': 'File saved to project storage.',
 'asset_id': '26aca4c0-82f5-4f6d-97e6-184079194cbb'}